In [1]:
import warnings

class Item:
    def __init__(self, count=0, max_count=64):
        self._count = count
        self._max_count = max_count
        
    def update_count(self, val):
        if val <= self._max_count:
            self._count = val
            return True
        else:
            return False

    @property
    def count(self):
        return self._count
    
    @property
    def max_count(self):
        return self._max_count

    def __add__(self, num : int):
        return self._count + num
    
    def __mul__(self, num : int):
        return self._count * num
    
    def __sub__(self, num : int):
        return self._count * num
    
    def __truediv__(self, num : int):
        return self._count/num

    def __iadd__(self, num : int):
        if self.count + num <= self.max_count and self.count + num >= 0:
            self._count += num
        else:
            warnings.warn(f"Count overflow. At {self}", UserWarning, stacklevel=2)
        return self
    
    def __imul__(self, num : int):
        if self.count * num <= self.max_count and self.count * num >= 0:
            self._count *= num
        else:
            warnings.warn(f"Count overflow. At {self}", UserWarning, stacklevel=2)
        return self
    
    def __isub__(self, num : int):
        if self.count - num <= self.max_count and self.count - num >= 0:
            self._count -= num
        else:
            warnings.warn(f"Count overflow. At {self}", UserWarning, stacklevel=2)
        return self
    
    def __itruediv__(self, num : int):
        if num != 0:
            self._count /= num
        else:
            raise ZeroDivisionError("Cannot be divided by zero")

    def __eq__(self, num : int) -> bool:
        return self.count == num
    
    def __ne__(self, num : int) -> bool:
        return not (self == num)
    
    def __gt__(self, num : int) -> bool:
        return self.count > num
    
    def __ge__(self, num : int) -> bool:
        return self > num or self == num
    
    def __lt__(self, num : int) -> bool:
        return not (self >= num) 
    
    def __le__(self, num : int) -> bool:
        return self < num or self == num

In [2]:
from enum import IntEnum

class Freshness(IntEnum):
    Maximum = 100
    Fresh = 70
    Rotten = 50
    Faded = 0

class Food(Item):
    def __init__(self,freshness=Freshness.Maximum, saturataion = 10, count=0, max_count=64):
        super().__init__(count, max_count)
        self._saturation = saturataion
        self._freshness = freshness
    
    @property
    def freshness(self):
        if self._freshness >= Freshness.Fresh:
            return "So fresh"
        if self._freshness >= Freshness.Rotten:
            return "Smell slightly"
        if self._freshness >= Freshness.Faded:
            return "It looks terrible"
    
    def eated(self):
        if self.count > 0:
            self - 1
            return (self._saturation * self._freshness/50) + 1
        return False

class Bread(Food):
    def __init__(self, freshness=Freshness.Maximum, saturataion=0, count=0, max_count=64):
        super().__init__(freshness, saturataion, count, max_count)
    
    def __str__(self):
        if self.count > 1:
            return f"{self.count} loafs of bread. {self.freshness}"
        return f"{self.count} loaf of bread. {self.freshness}"

class Ham(Food):
    def __init__(self, freshness=Freshness.Maximum, saturataion=0, count=0, max_count=64):
        super().__init__(freshness, saturataion, count, max_count)
    
    def __str__(self):
        if self.count > 1:
            return f"{self.count} pieces of ham. {self.freshness}"
        return f"{self.count} piece of ham. {self.freshness}"



In [3]:
class Maturity(IntEnum):
    Maximum = 100
    Mature = 80
    Half_mature = 50
    Immature = 0

class Fruit(Food):
    def __init__(self, maturity=Maturity.Immature, freshness=Freshness.Maximum, saturataion=0, count=0, max_count=64):
        super().__init__(freshness, saturataion, count, max_count)
        self._maturity = maturity
        
    @property
    def maturity(self):
        if self._maturity >= Maturity.Mature:
            return "ripe"
        if self._maturity >= Maturity.Half_mature:
            return "half - ripe"
        if self._maturity >= Maturity.Immature:
            return "unripe"
    
    def eated(self):
        return super().eated() * (self._maturity/100 + 0.1)

class Apple(Fruit):
    def __init__(self, color="red", maturity=Maturity.Immature, freshness=Freshness.Maximum, saturataion=0, count=0, max_count=64):
        super().__init__(maturity,freshness, saturataion, count, max_count)
        self._color = color
    
    def __str__(self):
        if self.count > 1:
            return f" {self.count} {self.maturity} {self._color} apples. {self.freshness}"
        return f" {self.count} {self.maturity} {self._color} apple. {self.freshness}"

class Banana(Fruit):
    def __init__(self,maturity=Maturity.Immature, freshness=Freshness.Maximum, saturataion=0, count=0, max_count=64):
        super().__init__(maturity, freshness, saturataion, count, max_count)
    
    def __str__(self):
        if self.count > 1:
            return f"{self.count} {self.maturity} bananas. {self.freshness}"
        return f"{self.count} {self.maturity} banana. {self.freshness}"

In [4]:
class Inventory:
    def __init__(self, max_count=100):
        self._max_count = max_count
        self._inventory_space = [None for i in range(max_count)]
    
    def __getitem__(self, index):
        return self._inventory_space[index]
    
    def __setitem__(self, index, item : Item):
        self._inventory_space[index] = item
        return self
    
    def __str__(self):
        res = "["
        for item in range(self._max_count):
            if self[item] != None:
                res += f"|{item}|: " + str(self[item]) + ", "
        res = res[0:-2] + "]"
        return res

    def put(self, index : int, item : Item):
        if self[index] == None:
            self[index] = item
            return True
        return False

    def throw_away(self, index : int):
        if self[index] != None:
            item = self[index]
            self.correct_slots()
            return item
        return None
    
    def use(self, index):
        if (isinstance(self[index], Food)):
            saturation = self[index].eated()
            self.correct_slots()
            return (Food, saturation)
        
    def correct_slots(self):
        for item in range(self._max_count):
            try:
                if self[item].count == 0:
                    self[item] = None
            except AttributeError:
                pass

In [5]:
apple = Apple(count = 2,freshness=Freshness.Faded, maturity=Maturity.Maximum)
banana = Banana(count = 2,freshness=Freshness.Maximum, maturity=Maturity.Half_mature)
bread = Bread(count = 2,freshness=Freshness.Rotten)
ham = Ham(count = 2,freshness=Freshness.Fresh)
print(f"{apple}\n{banana}\n{bread}\n{ham}\n")

inv = Inventory()
inv.put(5, apple)
inv.put(6, apple)
inv.put(7, apple)
inv.put(8, apple)
inv.put(9, apple)
inv.put(1, banana)
inv.put(2, bread)
inv.put(3, ham)
print(inv)
inv.use(5)
inv.use(5)
inv.throw_away(1)
print(inv)


 2 ripe red apples. It looks terrible
2 half - ripe bananas. So fresh
2 loafs of bread. Smell slightly
2 pieces of ham. So fresh

[|1|: 2 half - ripe bananas. So fresh, |2|: 2 loafs of bread. Smell slightly, |3|: 2 pieces of ham. So fresh, |5|:  2 ripe red apples. It looks terrible, |6|:  2 ripe red apples. It looks terrible, |7|:  2 ripe red apples. It looks terrible, |8|:  2 ripe red apples. It looks terrible, |9|:  2 ripe red apples. It looks terrible]
[|1|: 2 half - ripe bananas. So fresh, |2|: 2 loafs of bread. Smell slightly, |3|: 2 pieces of ham. So fresh, |5|:  2 ripe red apples. It looks terrible, |6|:  2 ripe red apples. It looks terrible, |7|:  2 ripe red apples. It looks terrible, |8|:  2 ripe red apples. It looks terrible, |9|:  2 ripe red apples. It looks terrible]


In [6]:
from dataclasses import dataclass, field
@dataclass
class MyQueue:
    _queue: list = field(default_factory=list)
    
    def __str__(self):
        return str(self._queue)

    def push(self, elem):
        self._queue.append(elem)
        return self

    def pop(self):
        item = self._queue[0]
        self._queue = self._queue[1:-1]
        return item
    
    def get(self):
        return self._queue.copy()

a = MyQueue([1,2,3,4,5])
print(a.pop())
a.push(6)
b = a.get()
print(b)
b[2] = 100
print(b, a)

1
[2, 3, 4, 6]
[2, 3, 100, 6] [2, 3, 4, 6]
